In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint

In [ ]:
nmon_metric_options = [
    ("timestamp", "Timestamp MS since Epoch", [0, 80], "timestamp"), #0
    ("CPU_ALL:User%", "CPU Total: User %", [0,100], "cpu_all_user_percent"), # 1
    ("CPU_ALL:Sys%", "CPU Total: System %", [0,100], "cpu_all_system_percent"), # 2
    ("MEM:memtotal", "Memory MB: Total", [0,65000], "mem_MB_total"), #3
    ("MEM:hightotal", "Memory MB: High Total", [0,65000], "mem_MB_high_total"), #4
    ("MEM:lowtotal", "Memory MB: Low Total", [0,65000], "mem_MB_low_total"), #5
    ("MEM:swapfree", "Memory MB: Swap Free", [0,1000], "mem_MB_swap_free"), #6
    ("MEM:active", "Memory MB: Active", [0,65000], "mem_MB_active"), #7
    ("MEM:inactive", "Memory MB: Inactive", [0,65000], "mem_MB_inactive"), #8
    ("NET:eno1-write-KB/s", "Network Write KB/s", [0,1000], "net_io_write_KBps"), #9
    ("NET:eno1-read-KB/s", "Network Read KB/s", [0,1000], "net_io_read_KBps"), #10
    ("DISKBUSY:sda", "Disk Busy %: /dev/sda", [0,100], "disk_io_busy_percent"), #11
    ("DISKREAD:sda", "Disk Read KB/s: /dev/sda", [0,10000], "disk_io_read_KBps"), #12
    ("DISKWRITE:sda", "Disk Write KB/s: /dev/sda", [0,10000], "disk_io_write_KBps"), #13
    ("VM:pgfault", "Paging and Virtual Memory: Page Faults", [0,100], "virt_mem_page_faults") #14
]

free_mem_metrics = [
    ("total", "Memory Total", [0,67000], "mem_total_MB"), # 0
    ("used", "Memory Used", [0,67000], "mem_used_MB"), # 1
    ("free", "Memory Free", [0,67000], "mem_free_MB"), #2
    ("shared", "Memory Shared", [0,67000], "mem_shared_MB"), #3
    ("buff/cache", "Memory Buffer/Cache", [0,67000], "buff_cache_MB"), # 4
    ("available", "Memory Available", [0,67000], "mem_available_MB") # 5
]

possible_experiments = [
    "tensorflow/neural_network/no_budgets/job_mode_multiprocessing",
    "tensorflow/neural_network/no_budgets/job_mode_multithreaded",
    "tensorflow/neural_network/no_budgets/job_mode_synchronous",
    "tensorflow/neural_network/static_budgets/sample_rate/10_percent",
    "tensorflow/neural_network/static_budgets/sample_rate/20_percent",
    "tensorflow/neural_network/static_budgets/sample_rate/50_percent",
    "tensorflow/neural_network/static_budgets/total_limit/10M",
    "tensorflow/neural_network/static_budgets/total_limit/20M",
    "tensorflow/neural_network/variance_budgets/10M_total_500_initial",
    "tensorflow/neural_network/variance_budgets/10M_total_1000_initial"
]

experiments = [
    "tensorflow/neural_network/no_budgets/job_mode_multiprocessing",
    "tensorflow/neural_network/static_budgets/sample_rate/20_percent",
    "tensorflow/neural_network/variance_budgets/10M_total_500_initial",
    "tensorflow/neural_network/static_budgets/total_limit/10M"
]

CB_color_cycle = {
    "blue":'#377eb8',
    "orange":'#ff7f00',
    "green":'#4daf4a',
    "pink":'#f781bf',
    "brown":'#a65628',
    "purple":'#984ea3',
    "gray":'#999999',
    "red":'#e41a1c',
    "yellow":'#dede00'
}

chosen_metric = nmon_metric_options[1]
metric_header = chosen_metric[0]
metric_name = chosen_metric[1]
y_axis_limits = chosen_metric[2]
metric_file_suffix = chosen_metric[3]

experiment_dfs = []  # net read

machine_names = []

for experiment in experiments:
    dataframes = []
    for i in range(0, 25):
        machine_id = f"lattice-{150+i}"
        filename = f"{i}-{machine_id}.nmon.csv"
        #filename = f"{machine_id}_free.csv"
        filepath = f"{experiment}/{filename}"
        dataframe = pd.read_csv(filepath, header=0)
        dataframes.append(dataframe.pop(metric_header))
    experiment_dfs.append((experiment, dataframes))

